# How to Run Inference on YOLOv7 with Native Pytorch and OpenVINO™ Torch-ORT

This tutorial is based on the [YOLOv7 repository](https://github.com/WongKinYiu/yolov7) by WongKinYiu. This notebook shows evaluation on **your own custom objects**. Many thanks to WongKinYiu and AlexeyAB for putting this repository together. 

### **Accompanying Blog Post**

We recommend that you follow along in this notebook while reading the blog post: [Accelerate PyTorch Models via OpenVINO™ Integration with Torch-ORT](https://blog.roboflow.com/accelerate-pytorch-openvino-torch-ort/)
<br>



### **Custom Dataset**

Follow [the getting started guide here](https://docs.roboflow.com/quick-start) to create and prepare your own custom dataset.


If you already have your own images (and, optionally, annotations), you can convert your dataset using [Roboflow](https://roboflow.com), a set of tools developers use to build better computer vision models quickly and accurately. 100k+ developers use roboflow for (automatic) annotation, converting dataset formats (like to YOLOv7), training, deploying, and improving their datasets/models.


### **Steps Covered in this Tutorial**

To run the inference on a test image we take the following steps:

* Evaluate YOLOv7 performance with Native Pytorch
* Evaluate YOLOv7 performance with OpenVINO™ integration with Torch-ORT

OPTIONAL:
* Reparameterize for Inference 
* Deployment
* Active Learning

#Install Dependencies

_(Remember to choose None in Runtime if not already selected. Runtime --> Change Runtime Type --> Hardware accelerator --> None)_

Reminder, uploaded files will get deleted when this runtime is recycled.

In [1]:
# Download YOLOv7 repository from Roboflow-ai and install requirements
!git clone https://github.com/roboflow-ai/yolov7.git
%cd yolov7
!pip install -r requirements.txt

fatal: destination path 'yolov7' already exists and is not an empty directory.
/content/yolov7
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached torch-2.0.0-cp39-cp39-manylinux1_x86_64.whl (619.9 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1
    Uninstalling torch-1.12.1:
      Successfully uninstalled torch-1.12.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch-ort-infer 1.13.1 requires torch==1.12.1, but you have torch 2.0.0 which is incompatible.


In [2]:
!pip install torch==1.12.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached torch-1.12.1-cp39-cp39-manylinux1_x86_64.whl (776.4 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0
    Uninstalling torch-2.0.0:
      Successfully uninstalled torch-2.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.15.1+cu118 requires torch==2.0.0, but you have torch 1.12.1 which is incompatible.
torchtext 0.15.1 requires torch==2.0.0, but you have torch 1.12.1 which is incompatible.
torchdata 0.6.0 requires torch==2.0.0, but you have torch 1.12.1 which is incompatible.
torchaudio 2.0.1+cu118 requires torch==2.0.0, but you have torch 1.12.1 which is incompatible.


# Download Correctly Formatted Custom Data

Next, we'll download our dataset in the right format. Use the `YOLOv7 PyTorch` export. Note that this model requires YOLO TXT annotations, a custom YAML file, and organized directories. The roboflow export writes this for us and saves it in the correct spot.

In [3]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="0a4WwzoVWX7OLQHdybod")
project = rf.workspace("a-s").project("uwh")
dataset = project.version(6).download("yolov7")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to UWH-6 in yolov7pytorch:: 100%|██████████| 1874/1874 [00:01<00:00, 1390.80it/s]


# Bring your own retrained model

If you have [retrained your yolov7 model on a custom dataset](https://blog.roboflow.com/yolov7-custom-dataset-training-tutorial/) from the Roboflow Universe, follow the steps in this section. 


**Note:** If you do not have retrained model, you can skip this section and use the pretrained model from ***content/yolov7/runs/best.pt***

In [4]:
# Mount your google drive to access the pretrained model obtained from https://blog.roboflow.com/yolov7-custom-dataset-training-tutorial 

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Evaluation

We can evaluate the performance of our custom training using the provided evalution script.

# Evaluate YOLOv7 performance with Native Pytorch

Note we can adjust the below custom arguments. For details, see [the arguments accepted by detect.py](https://github.com/WongKinYiu/yolov7/blob/main/detect.py#L154).

There are 2 minor changes we've made to run "detect.py" on CPU:
1.  We have commented out lines 38 and 39 in detect.py as the code is using [jit trace](https://pytorch.org/docs/stable/generated/torch.jit.trace.html).
2. Added lines 84 and 85 to enable device type as "cpu"

In [5]:
# If you DO NOT have a retrained model, use this command to run evaluation with the provided trained model: 

!python detect_without_jit.py \
 --weights /content/yolov7/runs/best.pt \
 --conf 0.25 \
 --img-size 640 \
 --source UWH-6/test/images/DJI_0021_mp4-32_jpg.rf.0d9b746d8896d042b55a14c8303b4f36.jpg

/usr/local/lib/python3.9/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.9/dist-packages/torchvision/image.so: undefined symbol: _ZN3c104impl8GPUTrace13gpuTraceStateE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
Namespace(weights=['/content/yolov7/runs/best.pt'], source='UWH-6/test/images/DJI_0021_mp4-32_jpg.rf.0d9b746d8896d042b55a14c8303b4f36.jpg', img_size=640, conf_thres=0.25, iou_thres=0.45, device='', view_img=False, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
YOLOR 🚀 3cf1e25 torch 1.12.1+cu102 CPU

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
Re

Refer to "Display Inference on test image" section to display the saved image.

In [6]:
# Run evaluation on a sample image with a retrained model

!python detect_without_jit.py \
 --weights /content/gdrive/MyDrive/TrainedModel/best.pt \
 --conf 0.25 \
 --img-size 640 \
 --source UWH-6/test/images/DJI_0021_mp4-32_jpg.rf.0d9b746d8896d042b55a14c8303b4f36.jpg

/usr/local/lib/python3.9/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.9/dist-packages/torchvision/image.so: undefined symbol: _ZN3c104impl8GPUTrace13gpuTraceStateE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
Namespace(weights=['/content/gdrive/MyDrive/TrainedModel/best.pt'], source='UWH-6/test/images/DJI_0021_mp4-32_jpg.rf.0d9b746d8896d042b55a14c8303b4f36.jpg', img_size=640, conf_thres=0.25, iou_thres=0.45, device='', view_img=False, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
YOLOR 🚀 3cf1e25 torch 1.12.1+cu102 CPU

Traceback (most recent call last):
  File "/content/yolo

# Display inference on test image

In [7]:
#display inference on a sample image

import glob
from IPython.display import Image, display

i = 0
limit = 10000 # max images to print
# Change the path below according to the location where result image is saved.
for imageName in glob.glob('/content/yolov7/runs/detect/exp/*.jpg'): #assuming JPG. 
    if i < limit:
      display(Image(filename=imageName))
      print("\n")
    i = i + 1

# Evaluate YOLOv7 performance with OpenVINO™ integration with Torch-ORT

In [8]:
# Install torch-ort-infer 
!pip install torch-ort-infer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Note we can adjust the below custom arguments. For details, see [the arguments accepted by detect.py](https://github.com/WongKinYiu/yolov7/blob/main/detect.py#L154).

Here, we added just 2 lines of code to boost performance with OpenVINO™ Torch-ORT

  line 17: from torch_ort import ORTInferenceModule

  line 71: model = ORTModule(model)   

In [9]:
# If you DO NOT have a retrained model, use this command to run evaluation using the provided trained model: 
!python detect_ort.py \
 --weights /content/yolov7/runs/best.pt \
 --conf 0.25 \
 --img-size 640 \
 --source UWH-6/test/images/DJI_0021_mp4-32_jpg.rf.0d9b746d8896d042b55a14c8303b4f36.jpg

/usr/local/lib/python3.9/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.9/dist-packages/torchvision/image.so: undefined symbol: _ZN3c104impl8GPUTrace13gpuTraceStateE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
Traceback (most recent call last):
  File "/content/yolov7/detect_ort.py", line 17, in <module>
    from torch_ort import ORTInferenceModule
  File "/usr/local/lib/python3.9/dist-packages/torch_ort/__init__.py", line 6, in <module>
    from onnxruntime.training.ortmodule import DebugOptions, LogLevel
ModuleNotFoundError: No module named 'onnxruntime.training'


In [10]:
# Run evaluation on a sample image with a retrained model 

!python detect_ort.py \
 --weights /content/gdrive/MyDrive/TrainedModel/best.pt \
 --conf 0.25 \
 --img-size 640 \
 --source UWH-6/test/images/DJI_0021_mp4-32_jpg.rf.0d9b746d8896d042b55a14c8303b4f36.jpg

/usr/local/lib/python3.9/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.9/dist-packages/torchvision/image.so: undefined symbol: _ZN3c104impl8GPUTrace13gpuTraceStateE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
Traceback (most recent call last):
  File "/content/yolov7/detect_ort.py", line 17, in <module>
    from torch_ort import ORTInferenceModule
  File "/usr/local/lib/python3.9/dist-packages/torch_ort/__init__.py", line 6, in <module>
    from onnxruntime.training.ortmodule import DebugOptions, LogLevel
ModuleNotFoundError: No module named 'onnxruntime.training'


# Display inference on test image

In [11]:
#display inference on sample image

import glob
from IPython.display import Image, display

i = 0
limit = 10000 # max images to print
for imageName in glob.glob('/content/yolov7/runs/detect/exp2/*.jpg'): #assuming JPG
    if i < limit:
      display(Image(filename=imageName))
      print("\n")
    i = i + 1

# Performance Boost with OpenVINO™ integration with TORCH-ORT

You may calculate the performance boost with OpenVINO™ integration with TORCH-ORT as compared to Native Pytorch using the formula:
<br>
</br>

PERFORMANCE BOOST (%) = 
(inference time with native pytorch - inference time with T-ORT / inference time with native pytorch) * 100


# Next steps

Congratulations, you've ran inference on the YOLOv7 model!
<br>
 Next, start thinking about deploying and [building an MLOps pipeline](https://docs.roboflow.com) so your model gets better the more data it sees in the wild.